# Geometric Harmonics

The Diffusion Map algorithm reduces the ambient dimension of a (finite) dataset $X$ sampled on a manifold by embedding it into the space of leading eigenfunctions of the Laplacian on the manifold. This can be used for dimension reduction of $X$ (as in diffusion_maps.ipynb) but also to *extend functions* which are defined on data $X$, i.e. $f(X): X \rightarrow \mathbb{R}$. Extending means that we are able to evaluate the function $f$ "outside" of $X$, e.g. in a neighborhood, for points $x_{new} \notin X$. We can, therefore, use Diffusion Maps for interpolation; even when the dimensionality of $X$ is large. The points $x_{new}$ are generally referred to as "out-of-sample", and more specifically for Diffusion Maps as the Nyström method/extension. The Nyström method allows to extend functions $f(X)$ using a special set of (basis-)functions defined on $X$, the so-called *geometric harmonics* (see [2], from page 49).

One application of the Nyström extension is to extend the eigenfunctions obtained from a subset of the data to the whole data set. The general (informal) workflow to describe the eigenfunctions on data is:

* Collect data $X \in \mathbb{R}^n $ which is of high dimension and expected to be of lower intrinsic dimension
* Apply Diffusion Maps: this gives us the new parametrization (=eigenvectors of kernel matrix) $\Phi \in \mathbb{R}^m$, such that $m < n$ (as in diffusion_maps.ipynb). Each of the $m$ vectors (i.e. evaluations of the eigenfunction) is a function defined on $X$ (which give a new parametrization in $\mathbb{R}^m$).
* We now obtain new data $X_{new} \notin X$ and we are interested to get the corresponding $\Phi_{new}$ values for $ X_{new})$. A straightforward approach is to re-compute all data $[X, X_{new}]$. This, however, has potentially unwanted side effects: the parameterization values of already reduced values $X$ change and the `epsilon` value may not be correct anymore. In the examples below, we use the *geometric harmonics* to extend the eigenfunctions on the data.     

#### Issues purposely left out to maintain a clear structure in the notebook:
* When applying Diffusion Maps for dimensional reduction, forward and inverse mapping (mappings are described below), there are three different (and optimal) `epsilon` values. These can be either set heuristically or can be optimized (e.g. using the residual for interpolation). More sophisticated methods are k-fold cross-validation.
* The number of eigenpairs can also be optimized. Usually, the more the better, however, there are also numerical issues at some point (see [2]).


[1] Yoshua Bengio et al., Out-of-Sample Extensions for LLE, Isomap, MDS, Eigenmaps, and Spectral Clustering, https://papers.nips.cc/paper/2461-out-of-sample-extensions-for-lle-isomap-mds-eigenmaps-and-spectral-clustering.pdf

[2] Stephane Lafon, Diffusion Maps and Geometric Harmonics, PhD thesis, Yale University, U.S.A., 2004

In [ ]:
import numpy as np

from sklearn.datasets import make_swiss_roll
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import pdist

from tutorial_progress import TutorialBar

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from skopt import gp_minimize
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3

# NOTE: make sure "path/to/datafold" is in sys.path or PYTHONPATH if not installed
import datafold.dynfold as dfold
import datafold.pcfold as pfold
from datafold.dynfold import GeometricHarmonicsInterpolator as GHI

random_state = 1

## 1. Step: Define functions on available data
We compute the eigenvectors (as a finite representation of eigenfunctions) and use them as a new parametrization (here, for the swiss roll example, using $\phi_1$ and $\phi_5$, as in diffusion_maps.ipynb). In the subsequent steps, we want to interpolate the eigenvectors $\phi_{1,5}$ outside the available data $X$.

In [ ]:
# obtain dataset with a lot of points to get accurate eigenfunctions
np.random.seed(random_state)
X_all, color = make_swiss_roll(n_samples=20000, noise=0, random_state=random_state)

num_eigenpairs=6

pcm = pfold.PCManifold(X_all)
pcm.optimize_parameters()

dmap = dfold.DiffusionMaps(epsilon=pcm.kernel.epsilon, cut_off=pcm.cut_off, n_eigenpairs=num_eigenpairs)
dmap = dmap.fit(pcm)
evecs, evals = dmap.eigenvectors_, dmap.eigenvalues_

# new parametrization, used as "ground truth" (see diffusion_maps.ipynb for why the 1st and 5th is used)
phi_all = evecs[:,[1, 5]]

# select a subset of the data to proceed, so that the next computations are less expensive
ind_subset = np.sort(np.random.permutation(np.arange(X_all.shape[0]))[0:2500])
X_all = X_all[ind_subset,:]
phi_all = phi_all[ind_subset,:]
color = color[ind_subset]

In [ ]:
# Plot:
fig = plt.figure(figsize=[10, 5])

fig.suptitle(f"total #samples={X_all.shape[0]}")

ax = fig.add_subplot(1, 2, 1, projection="3d")
ax.set_title("all data, original coordinates")
ax.scatter(*X_all.T, s=5, c=color, cmap=plt.cm.Spectral)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

ax = fig.add_subplot(1,2, 2)
ax.set_title("all data, diffusion map coordinates")
ax.scatter(*phi_all.T, s=5, c=color, cmap=plt.cm.Spectral);
ax.set_xlabel(r'$\phi_1$')
ax.set_ylabel(r'$\phi_5$');

## 2. Step: Setting up data for geometric harmonic interpolation 

We split the data $X$ and target values $\Phi$ (computed in step 1) into training and test set. For the training set we have $\{X_i, f(X_i)\}_{i=1}^{N_{train}}$ for randomly sampled $X$ values. Later, we want to find the corresponding $f(X_j^{(test)}) = \phi_j^{(1,5)} $. We compute the Diffusion Map coordinates on the training data, which are required for the interpolation.

Remarks:
* 1: The Diffusion Map computed on $X^{(train)}$ for interpolation are used as a (functional) basis. This means we take the eigenvectors ordered by the corresponding eigenvalue and we do not need to select the coordinates as for the dimensional reduction
* 2: it is highly recommended to *not* use a normalized kernel (larger errors were observed in the experiments)
* 3: in the code (folder `dm`), *only* the sparse version of Diffusion Maps works currently for *GeometricHarmonicInterpolator*. To have no side-effects, the `cut_off` is set to infinity to mimic the case of dense evaluation.
* 4: for the interpolation basis, we usually need more eigenpairs than for dimensional reduction (here, 100 instead of 2)
* 5: from point 4 follows that the epsilon values are more suitable at larger scales (in the example: 100 for interpolation vs. 1.25 for dimensional reduction) 
* 6: the eigenvalues behave $\lambda_i \rightarrow 0, \text{ for } i \rightarrow \infty$. The interpolation using geometric harmonics involve a term $1/\lambda_i$. This means, with too many eigenpairs, instabilities/numerical issues show up.

In [ ]:
# we use a random subset of the data and parametrization repectivly
# the phi_test values are used for the "ground truth" values to measure an error
# the color values are used for plotting
X_train, X_test, phi_train, phi_test, color_train, color_test = train_test_split(X_all, phi_all, color, train_size=2/3)

# compute the geometric harmonics from X to phi
num_eigenpairs = 100

# estimate resonable epsilon and cut_off values automatically
pcm = pfold.PCManifold(X_train)
pcm.optimize_parameters()

# construct the GeometricHarmonicsInterpolator and fit it to the data.
gh_interpolant = GHI(epsilon=pcm.kernel.epsilon**2, cut_off=pcm.cut_off,
                     n_eigenpairs=num_eigenpairs, symmetrize_kernel=False)

gh_interpolant.fit(pcm, phi_train);

## 3. Step: Compute the interpolation functions
Use geometric harmonics from previously computed Diffusion Map. These can be used as geometric harmonics for interpolation.

### Compute residual of interpolation functions:

In [ ]:
def compute_error(tr, ap):  # also used later to compute the error
    return np.sqrt(np.mean((tr-ap)**2))

phi_train_approx = gh_interpolant.predict(X_train)
res0 = compute_error(phi_train_approx, phi_train)

phi_test_approx = gh_interpolant.predict(X_test)
res1 = compute_error(phi_test_approx, phi_test)

print(f"residual train: {res0}, residual test: {res1}")

## 4. Step: Evaluate interpolation function

On the left side, we plot the "ground truth" function values. On the right side, we evaluate the interpolation functions.

In [ ]:
fig,ax = plt.subplots(1,2,figsize=[10, 5],sharey=True)

ax[0].set_title("ground truth test set ${\phi}_{1,5}$")
ax[0].scatter(phi_test[:, 0], 
           phi_test[:, 1], s=10, c=color_test, cmap=plt.cm.Spectral)
ax[0].set_xlabel(r'$\phi_1$')
ax[0].set_ylabel(r'$\phi_5$')

ax[1].scatter(*phi_test.T, s=10, c=color_test, cmap=plt.cm.Spectral)
ax[1].set_title(r"interpolated values $\hat{\phi}_{1,5}$");
ax[1].set_xlabel(r'$\hat{\phi}_1$')
ax[1].set_ylabel(r'$\hat{\phi}_5$');


## 5. Step: Measure and plot error

In [ ]:
error1 = compute_error(phi_test[:, 0], phi_test_approx[:,0]) 
error2 = compute_error(phi_test[:, 1], phi_test_approx[:,1])
print(f"error phi0: {error1}, error phi1: {error2}")

error_color = (phi_test[:, 0]-phi_test_approx[:,0])**2 + (phi_test[:, 1] - phi_test_approx[:,1])**2

In [ ]:
fig,ax = plt.subplots(1,2,figsize=[12, 5],sharey=True)

sc = ax[0].scatter(phi_test[:, 0], phi_test[:, 1], c=error_color, cmap="Reds")
ax[0].set_title("test data with error (absolute)")
plt.colorbar(sc,ax=ax[0]);
ax[0].set_xlabel(r'${\phi}_1$')
ax[0].set_ylabel(r'${\phi}_5$')

# the np.newaxis need are required to have 2D arrays:
norm_factor = np.max([np.max(pdist(phi_test[0, :][:, np.newaxis])), 
                      np.max(pdist(phi_test[1, :][:, np.newaxis]))])   # take max. distance in test dataset as the norming factor

sc = ax[1].scatter(phi_test[:, 0], phi_test[:, 1], vmin=0, vmax=.1, c=error_color/norm_factor, cmap="Reds")
plt.colorbar(sc,ax=ax[1]);
ax[1].set_title(f"test data with error (relative)");
ax[1].set_xlabel(r'${\phi}_1$')
ax[1].set_ylabel(r'${\phi}_5$');

## 6. Step: Geometric Harmonics to interpolate inverse of dimensional reduction transformation

This step basically repeats the above steps (except step 1, we use the same data samples), however, this time we want to interpolate $X$ values defined on $\Phi$ values. This is $f^{-1}: \Phi \rightarrow X $ and therefore we obtain the inverse mapping of the dimensional reduction (carried out in step 1). We have to compute another Diffusion Maps, but this time on the $\Phi$ values as a dataset. We then interpolate all original coordinates as function values on $\Phi$. There are three dimensions in the original space of the swiss roll, so we need three interpolation functions.

* Because the Euclidean distances are much smaller in the reduced $\Phi$ space, we need a much smaller `epsilon` value.


In [ ]:
# shuffle again new sets.
# NOTE: this time, the "ground truth" values are X_test (-> target values)

print('Generating test/train split...')
X_train, X_test, phi_train, phi_test, color_train, color_test = train_test_split(X_all, phi_all,\
                                                                color, train_size=2/3, random_state=random_state)

print('Computing bounds for parameters...')
pcm = pfold.PCManifold(phi_train) # (!!) we use the dmap space as base space now, and transform to X
pcm.optimize_parameters(random_state=random_state)

opt_epsilon = pcm.kernel.epsilon
opt_cutoff = pcm.cut_off
opt_n_eigenpairs = 100

# test the interpolation quality without optimization (which is the next cell)

gh_interpolant_phi_to_X = GHI(epsilon=opt_epsilon,\
            cut_off=opt_cutoff, n_eigenpairs=opt_n_eigenpairs,\
            symmetrize_kernel=True, is_stochastic=False)
gh_interpolant_phi_to_X.fit(phi_train,X_train)

res0 = compute_error(X_test, gh_interpolant_phi_to_X.predict(phi_test))
print(f"residual without further parameter optimization: {res0}")

# plot ground truth and interpolated values
fig = plt.figure(figsize=[16, 9])

ax = fig.add_subplot(121, projection="3d")
ax.scatter(X_test[:, 0], X_test[:, 1], X_test[:, 2], c=color_test, cmap=plt.cm.Spectral)
ax.set_title("ground truth values")
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
ax.set_zlabel(r'$x_3$')

ax = fig.add_subplot(122, projection="3d")
ax.scatter(*(gh_interpolant_phi_to_X.predict(phi_test)).T,
           c=color_test, cmap=plt.cm.Spectral)
ax.set_title("interpolated values (no hyperparameter optimization)");
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
ax.set_zlabel(r'$x_3$');

In [ ]:
n_iters = 10

np.random.seed(random_state)

# Bayesian optimization using Gaussian processes with skopt.
# Note that GHI scores are the mean squared error, so we want to maximize the negative score.
# Also note that cross-validation is tricky with kernel methods, as reducing the data set also
# changes which parameters are "optimal". Here, we use the entire training set to find the parameters.
opt = BayesSearchCV(
    GHI(symmetrize_kernel=True, is_stochastic=False),
    {
        'epsilon': Real(pcm.kernel.epsilon/5, pcm.kernel.epsilon*5, prior='log-uniform'),
        'cut_off': Real(pcm.cut_off/2, pcm.cut_off*2, prior='uniform'),
        'n_eigenpairs': Integer(100, 1000, prior='uniform'),
    },
    n_iter=n_iters,
    random_state=0,
    scoring=lambda estimator,x,y: -np.sum(estimator.score(x,y)),           # is to be maximized
    cv=[[np.random.permutation(X_train.shape[0]),                          # train indices
         np.random.permutation(X_train.shape[0])[0:X_train.shape[0]//10]]] # test indices
)

# display a progress bar
bar = TutorialBar(n_iters)

# run the optimization
opt.fit(phi_train, X_train, callback=lambda info: bar.update())

# get the results
optimal_GHI = opt.best_estimator_
print(f'Previous epsilon: {pcm.kernel.epsilon}, cut-off: {pcm.cut_off}, #eigenpairs: {num_eigenpairs}')
print(f'Optimal epsilon: {optimal_GHI.epsilon}, cut-off: {optimal_GHI.cut_off}, #eigenpairs: {optimal_GHI.n_eigenpairs}')

In [ ]:
# compute residual for best model
res0 = compute_error(X_test, optimal_GHI.predict(phi_test))

print(f"residual X0: {res0}")

In [ ]:
# plot information about the Geometric Harmonics (i.e. eigenvalues and eigenvectors)
fig,ax=plt.subplots(1,1)
ax.semilogy(optimal_GHI.eigenvalues_,'.-')
ax.set_xlabel('#eigenvalue')
ax.set_ylabel(r'$\lambda$')

fig,ax = plt.subplots(2,6,figsize=[10,3],sharex=True,sharey=True)
for k1 in range(ax.shape[0]):
    for k2 in range(ax.shape[1]):
        index = k2 + k1*ax.shape[1]
        ax[k1,k2].scatter(*phi_train.T,s=10, c=optimal_GHI.eigenvectors_[:,index], cmap=plt.cm.Spectral)
        ax[k1,k2].set_title(r'$\phi_{' + str(index) + '}$')
fig.tight_layout()

In [ ]:
# plot ground truth and interpolated values
fig = plt.figure(figsize=[16, 9])

ax = fig.add_subplot(121, projection="3d")
ax.scatter(X_test[:, 0], X_test[:, 1], X_test[:, 2], c=color_test, cmap=plt.cm.Spectral)
ax.set_title("ground truth values")
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
ax.set_zlabel(r'$x_3$')

ax = fig.add_subplot(122, projection="3d")
ax.scatter(*(optimal_GHI.predict(phi_test)).T,
           c=color_test, cmap=plt.cm.Spectral)
ax.set_title("interpolated values");
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
ax.set_zlabel(r'$x_3$');

In [ ]:
# compute and plot error
error1 = compute_error(X_test[:, 0], optimal_GHI.predict(phi_test)[:,0]) 
error2 = compute_error(X_test[:, 1], optimal_GHI.predict(phi_test)[:,1])
error3 = compute_error(X_test[:, 2], optimal_GHI.predict(phi_test)[:,2])
print(f"error X0: {error1}, error X1: {error2}, error X2: {error3}")

error_color = np.linalg.norm(X_test - optimal_GHI.predict(phi_test),axis=1)

fig = plt.figure(figsize=[16, 7])

ax = fig.add_subplot(121, projection="3d")
sc = ax.scatter(X_test[:, 0], X_test[:, 1], X_test[:, 2], c=error_color, cmap="Reds")
plt.title("test data with error (absolute)")
plt.colorbar(sc);
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
ax.set_zlabel(r'$x_3$')

ax = fig.add_subplot(122, projection="3d")
# the np.newaxis need are required to have 2D arrays:
norm_factor = np.max([np.max(pdist(X_test[0, :][:, np.newaxis])), 
                      np.max(pdist(X_test[1, :][:, np.newaxis])), 
                      np.max(pdist(X_test[2, :][:, np.newaxis])) ])   # take max. distance in test dataset as the norming factor

sc = ax.scatter(X_test[:, 0], X_test[:, 1], X_test[:, 2], vmin=0, vmax=.1, c=error_color / norm_factor, cmap="Reds")
plt.colorbar(sc);
plt.title(f"test data with error (relative)");
ax.set_xlabel(r'$x_1$')
ax.set_ylabel(r'$x_2$')
ax.set_zlabel(r'$x_3$');